In [1]:
%reset -sf

In [2]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
# d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [10]:
srr = """
3344
8938
7923
3979
2753
5730
4225

24216
7432
18284

3475
9177
6769
11335
8061
9302
8132

25796

20937

2039
4989
5932
5434
5088
7185
1271
8359
5489
8646

7796
1233
3529
4118
7093
7090
8757
2283
5562
3789

2296
2105
1978
2981
5682
1627
2285
5654
5596
2022
2733
3078
3176
4158

6294
3789
1863
5121
6122
7309
7063
4233
1717
6755
5505
2713

4988
5133
4933
6407
2897
2682
5328
4268
1518
4631
2736
1432
3812
1031

4553
1109
7214
2744
8389
3180
7464
5584
7689
6452

5645
6207
2497
1114
5747
2620
2621
1866
2334
1324
5879
6322
4954

8723
24316
4375

4051
6866
7417
4650
6035
6580
3881
3041
1443
6659
2951
2387

5606
1862
3300
6106
5944
2306
1935
5717
6204
1882
2455
4790
4534
4144

3278
5536
4686
4331
1750
1742
2103
6754
5417
2242
2374
5133
2315

34403

5574
2129
2846
5790
1428
1220
1523
5768
2226
1442
1139
3295
3831

3231
3998
5904
6878
3082
2159
2806
2388
1878
1393
6929
6812

18094

4972
4575
5522
5582
2776
3971
3595
1959
4161
4550
1987
4519
4551
3546
3562

4913
5328
2726
5349
6670
7650
7799
3818
5536
2765

1674
3454
5921
6170
3474
1401
5993
5098
3269
5671
5385
5418

4381
35451

1970
6227
6781
3622
2921
2360
5673
2344
5503
5006
6866
1804
2559

12473
1374
23850

7943
1442
3091
2907
1538
6027
1389
4851
3301
1324
6017

13712
11949
4399
1093
2546
6876

5429
1453
7989
9605
2344
10505
4563
3537

6061
1428
3435
5752
1097
3313
2318
1656
2206
6922
6612
2805
3123

2613
6493
3283
5206
4118
3058
1377
4453
6195
3371
5520
6197
5121
1622

8199
5761
6985
6058
8277
9247
8937
4739
5107

11764
13344
5462

4453
10616
8770
6014
6991
7898

5873
12987
16138

9294
8381
3175
1945
8195
2851
1780
6140

3655
3450
6771
4950
3491
6883
4408
4505
2513
6768
4868
3883
4458

39217

15599
1459
19930
6992

9392
6380
4470
5228
8009
7680
1815
4365
5529

8677
6381
10150
2299
2071
9364
5633

1778
5304
1883
2215
4288
4214
7602
2987
7011
1652
5977

2093
20240
18383

4600
2704
3915
3853
5659
2905
3480
3546
4647
6062
3613
6095
4079

6352
4362
5767
7708
2318
10738
1956

5581
6478
5624
3850
5025
3754
2559
4730
1299
1187
3771
1432
1590
5108

3412
10022
1478
3654
4938
3611

3455
5118
6647
7160
7244
2902
6305
1204
2124
5595
7040
5145

1502
8426
6808
9175
3902
5516
5310
4122
7357

8679
14080

23992
17036

1880

2799
3910
3661
3680
4159
5746
6249
2614
2012
7433
5092
3067

35016
24321

9575
2970
17157

3129
2954
2520
5998
2854
5823
2469
3786
2683
3300
3360
5413
3959

28905
4100

24304
7180
23566

5990
1740
1955
5498
1754
3458
4856
4499
3560
3275
1443
1815
1511
1106

1466

10351
8066
5699

4995
7451
7189
2348
3600
3320
3351
5857
6388
3787
1923
6228

6125
1302
6779
3279
13428
7624

41865

2353
11924
6498
10636
1592
3990
2269

7281
4294
3083
10444
4489
4468
1400
4482

63998

2373
2091
6067
3372
1996
6542
1423
5041
6251
4682
5574
2506
6784

1243
1014
1653
2960
1459
4820
4623
5541
3455
3260
1161
1395
3963

3299
1686
4068
7455
3737
6857
7771
7383
7769
4641
3105

8109
1367
5553
8315
8042
4244
6974
1062

50228

6657
22682

4093
5296
8752
9532
11488
3792
11225

2753
16207
10646
11950

1505
4569
6688
4674
7154
7342
4675
8434
5637
3789

5625
3177
2820
1075
3657
4862
3879
1954
3185
4563
1413
5323
6048
3166

4933
1874
5953
6528
6915
3449
6867
6892
6105
3012
2055
5516
3026

14676
25751
4615

3163
5888
4144
3845
6817
6989
6211
1620
7624
1390
5321

8348
2441
12492
12656
13889

6433
1032
2005
5844
3864
2370
6664
5540
2943
5135
3375
2236
4305

6269
3979
1844
3799
3947
5858
1338
2761
2285
5192
1358
1810
5009
1346

4183
3118
4232
5655
4391
4210
5644
3865
4049
1951
2839
5212
1862
5991
4452

4299
1037
3491
3002
6062
2026
1007
7398
1697
5797
5922
5352

53946

7483
3663
1134
4279
5315
2756
1280
5807
1892
3701
2848

21603
4098
21937

1914
1812
6990
2754
2459
5984
3895
7411
5240
1022
4323
4382

3803
1698
5774
1680
2122
2865
6470
2921
7619
4584
1788

28678
10385

12207
12265
3932
13284
12672

5559
5610
1230
5547
3945
1566
2589
1948
2313
4243
2472
1343
1317
5152
1953

1082
6063
1974
2054
2931
5433
4410
4584
4896
2825
3945
5849
3615
2464
1560

6494
7717
4697
3012
4515
3802
8621
6041
6939
3059

1747
8291
4058
2884
1775
8465
5889
4807

6263
7346
8014
8081
5361
4950
2721
5656
3858
7486

11877
5145
10149
2148

8389
4492
5074
9457
5132
4355
3254
9607
4059

5858
7062
9260
6672
2291
1448
7436

8943
6577
8042
2056
9801
9060
10243
10758

11606

6340
4468
4225
6861
5648
2951
2556
5794
2657
1450
5888
4363

1294
3826
3132
2798
5133
2118
4319
1681
5402
4255
4831
1300

4233
6221
6336
5287
5566
2796
3047
5486
1849
3572
3709
7014

2418
7218
1153
8036
2015
1385
5385
2775

5788
17132
1734
6843

17843
2755
4097
1314

7448
3025
3255
6379
1817
7363
4086
3106
7192
3632
2748
6634

11495
6431
6756
7335

3090
2890
6300
5939
4045
1781
5080
6123
2589
5990
4438
2903
2789
4761

2633
5069
6129
1870
2148
2688
4932
5756
5281
3209
2040
5708
2292
2027

68293

4087
4630
5781
5165
3022
1847
4922
2792
4629
6178
3427
4536
4802
1733

11367
12689
13239

7698
7879
2401
7045
9018
2767
8286
5754
7154

43194

4845
3694
1107
3687
7597
7787
8056
8010

3598
4505
3676
5174
4271
1290
1116
5257
1343
2660
6227
1303
1217

31264
17846

3026
3533
4579
4017
3465
5489
3720
3146
1755
6197
2181
6449
4893
1766

1225
2725
5173
3401
4172
2673
6005
8950
8838

7416
1467
4741
2043
3032
2290
4182
4493
4860
4543
6791

9989
23051
12756

2995
1859
2029
1520
1287
5281
2299
2794
6913
1477
3639
5475
1198

4513
5732
6462
1327
1149
6276
5925
5654
5283
1177
1802
2093
4928
1355

8874
2550
10268
8362
9158
6929
10674

5052
2678
6754
6411
1197
7463
3835
5308
8160

9428
1308
1957
6146
7962
9350
3699
6982
7111

5388
3451
3865
4881
5709
6812
3177
2124
3329
8718

16156
14022
16031
7972
2381

3696
12103
9136
9793
11491
6030

10542
9283

1040
3593
1510
2450
5138
5137
1935
3159
2562
1604
5280
1282
1059
3784
2287

2464
4363
6262
1136
7072
2328
1517
7229
5730
2456
1681
7213

4983
4935
6580
1118
5291
4664
4778
6818
5331
6254
5525
1214
1659

4535
4602
1741
2803
2795
1195
2041
1147
5091
4244
6115
4765
2074
4711
2985

1487
5322
2729
5893
4743
5788
5701
5954
1570
2214
7415
2549

7483
12474
7468
5340

1699
1294
4841
3882
1746
5642
5753
4051
2750
4079
5881
4307
2535

36712
25729

1493
9804
6491
3737
5900
8810
2677

32912
35589

15457
17689
15413

9642
12851
7416
5456

10946
9357

34589

11079
18789

1029
13280
1556
6080
14840

2352
7024
1303

6432
4312
3800
1713
4580
2574
6955
6129
1054
4104
4150
3146
1777

8253
6799
1492
8292
6260
4014
7500
3738
1349
7391

9308
8918
4943
1408
6212
7375
8604
6907

3994
10181
7867
4339
8321
3496
6332
8291

5058
6311
8770
10517
8510
1807
10295
2060

11864
12981
10287
10881

17316
9524
16372

4423
5322
4954
11336
8543
8031

1576
9103
8531
11398
4900
1621

5162
1057
12960
16173
8403

2436
4121
5143
3486
5542
2598
6157
5536
6005
1953
2645
1790

1211
5673
7658
6331
6354
5775
2821
5983
5338
6135
7436

3891
4230
6457
5382
8435
8484
6872
1750
6127

8997
7290
6789
13725
3285
13318

7602
6264
6286
5290
4403
3829
5456
2632
2556
5225
1189

9054
11332
1990
6952
3916
10222
1268

3599
4346
2535
2775
1140
5020
1050
1119
1373
6191
6996
7381

15115
5092

5201
4358
6447
3800
5425
2062
6314
3184
5203
5626
6024
1408
2543
5400

4602
8699
3729
3608
3425
6182
1210
4268
8660

5167
3056
2162
5024
5586
7900
4688
2166
7268
2577

2871
4288
3167
7065
7042
6419
2788
6489
1584
3162
5541

8083
9590
8100
2032
6749
4610
4924
2628
2110

2103
5752
2489
2898
5077
4422
1491
5660
2916
6302
5032
1998
5007
3953

7090
2458
15291
7943
4677

3938
6524
3715
1700
3637
3124
6519
5349
1768
5571
1725
1038
4694

19691

6409
2963
4723
3601
6292
1298
4875
4930
5750
6376
4620
5066
6434
5973

7194
10283
4302
6346
6744
4990
9302
3257

5500
4929
4011
3026
1247
5017
2477
2815
3206
3386
5563
3704
2510
5969

3588
4251
1214
4568
4747
4950
4670
3810
4616
1599
5738
2651
5971

7252
4958
7736
7267
1115
1936
3503
8340
3581

19911
9923
5937
11827

16111
1929
1567
2584

4411
6070
5904
5425
1684
2343
1569
4476
3774
5901
2230
4560
4894
3532
3679

19875
3275
1303

1111
2927
1812
2341
5611
1602
1876
2430
2810
5400
1909
1341
2605
1860

1387
4755
1827
3107
5734
3912
7449
5100

12046
4731
10977
8509
15801

9891
19111
15136
6515

5839
2497
4195
2027
5340
6432
7256
2774
4934
6521
4715

5351
6545
5531
10516
10280
10482
6863

54895

3515
3873
1035
9079
10474
8983
6396

1564
5340
2239
2539
2464
1083
3777
2981
4247
4471
1687
1693
3053
5160
3384

4827
3949
3489
5192
3342
4695
4461
6165
1891
5060
5961
1543
2959
6262

18480
2939
10591

3496
14216
4695

2178
5400
5298
6806
2108
3793
5762
4787
3032
3819
1834
2905
5070

8889
1553
11822
11140
1604
11187
3094

3950
3808
6643
1323
4637
1465
5416
6504
5400
6691
6056
4358
5209

5821
4259
2389
7085
6813
2094
2901
7089
6160
6301
4980
4239

5431
5016
9948
1158
3133
4783

2650
6541
6230
6789
6499
2613
2358
3241
3777
5018
4522
4057
5587

3428
5879
2228
2436
5905
4346
6072
3843
2432
1342
1339
4727
6823

15873
19800
13068

1866
12061
11047
15922

19221
9883
5634

1072
13511
18294

64548

5313
13972
2829
16260

2945
2469
1966
2968
4563
3434
3095
5102
5678
2491
3625
5366
2994

4920
3123
4726
5313
3148
3510
5397
2081
5252
1949
4929
6472
5292

6040
3246
6110
4395
2589
6227
4105
3881
5992
4294

6363
3364
7225
7105
1912
7460
1929
2680
6231
3061
7856

5043
1059
4183
4697
3634
3572
1636
2635
2733
5600
4000
6300
1367

2560
1074
1473
6555
6887
5044
6544
5101
3041
3100
2924
5388
6025

8736

2235
19236
12524

3968
2552
11328
10347
5092
4822

3051
6914
7082
4322
4701
5313
1889
5562
7529
7673
3085

1518
10390
1706
5313
7800
8948

6670
6555
1881
6658
5687
1338
1547
6876
4657
2061
3742
2958
6836

9112
2305
9461
10472
8796
8858
5303
3259

4281
1131
2165
10304
9662
9627
10574

8949
12010
13033
5232
3433
8212

8478
9908
9043
4869
11600
12466

2444
8308
9337
3539
6796
7175
4099
9456
6075

2503
1993
3162
9282
3378

4406
1417
10195
6361
3735
10131
4131
2617

28770

11378
13601
14324

9222
1090
2246
3436
4692
2317
6684
5515
3608

8442
8781
6388
6511
5283
3505
7294
3703
2752
2541

2160
5683
8706
7818
1283
3659
1465
7255
2556
7496

4351
2585
3091
6105
2960
6796
6028
1223
3791
2543
2221
1342
1228

2678
5749
8275
6535
4567
2629
5222
8406
3528
7274

11561
8495
11196
6072
4726
6342
4271

3035
4638
6716
4871
5516
2387
1828
3695
5199
4602
2372
5376
6229

5904
7241
1735
4137
5267
7228
6840
2122
5707
3875
5913
6192

3372
6579
6596
4463
2498
3147
1505
1879
5437
6317
2129
3425
1569

7582
4017
9553
5929

2721
1439
5723
1790
10437
10251
6175

3037
2342

4409
7389
6107
1920
5975
3859
5681
7411
2409
5083
2678
3001

6304
6453
8158
1532
2956
1314
4902
5140

1964
6243
8353
3374
7134
1951
2425
6063
2765

7148
12369
7870
3802
10446
10955

1377
2579
4886
4537
5929
2946
3894
5212
3825
1003
1884
5017
4114
1895
""".strip()

# with open("/Users/hkmac/Downloads/input.txt") as f:
#     srr = f.read().strip()

brr = srr.split("\n\n")
brr = [sum(int(x) for x in srr.split("\n")) for srr in brr]

In [15]:
sum(sorted(brr)[-3:])

206104

In [ ]:
def test(arr):a
    res = 0
    return res